## Convolutional Neural Network Intro - Deep Learning with PyTorch 10

![alt text](59954intro_to_CNN.jfif "A typical CNN model")

## Image Filter / Image Kernel Overview - Deep Learning with PyTorch 11

![alt text](image.png "Conversion of image to Matrix")

![alt text](convolutional-layer.jpg "Convolutional layer")

Image kernel is creating a little matrix and dragging it all over the image

Image kernels explained visually: https://setosa.io/ev/image-kernels/

## Convolutional Layer and RGB - Deep Learning with PyTorch 12